In [ ]:
from astropy.io import ascii
from glob import glob
import warnings
import numpy as np
from astropy import nddata, table, stats
from scipy import ndimage
import os
from astropy.io import fits, ascii


In [ ]:
calfile = glob('/*.fit', recursive=True)
rawfile = glob('raw/2/*.fit', recursive=True)

print('{} uncalibrated images'.format(len(rawfile)))
print('{} calibrated images'.format(len(calfile)))

In [ ]:
from glob import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pds4_tools import read

calxml = glob('/*06.xml')

for name in calxml:
    prods = read(name)

    # Grab the 3D image cube by its structure name
    cube = prods['image'].data  # masked array, shape: (bands, lines, samples)

    # Choose a display plane (e.g., first band) or collapse bands
    if cube.ndim == 3:
        img = cube[0]  # or: np.nanmedian(cube, axis=0)
    else:
        img = cube

    # Replace mask with NaN for safe percentile stretch
    img = np.ma.filled(img, np.nan)

    # Simple contrast stretch to 1–99th percentile
    vmin, vmax = np.nanpercentile(img, [1, 99])

    plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    out = Path(name).with_suffix('.png')
    plt.axis('off')
    plt.savefig(out, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()


In [ ]:
calxml

In [ ]:
#

import pds4_tools
calxml = glob('MVIC/data_dinkinesh_raw/*.xml', recursive=True)
#rawxml = glob('*.xml', recursive=True)

import matplotlib.pyplot as plt
for name in calxml:
    structures = pds4_tools.read(name) 
    image = structures[2].data
    plt.imshow(image, cmap = 'gray')
    fname = name+'.png'
    plt.savefig(fname, dpi = None, facecolor = 'w', orientation = 'portrait', format = 'png')

In [ ]:
import pds4_tools
xml = glob('/*.xml', recursive=True)

In [ ]:
xml

In [ ]:
pds4_tools.view('*.xml')
#pds4_tools.view(xml[11])

In [ ]:

import warnings
import numpy as np
from astropy import nddata, table, stats
from scipy import ndimage
import os
from astropy.io import fits, ascii
class ImageSet():
    """Convenience class to organize a set of images.

    Attributes
    ----------
    .image : 3D or higher dimension array of shape (..., N, M), images
        to be centroided.  The shape of each image is (N, M).
    .file : string array, FITS file names of images to be centroided
    .loader : function
        Image loader to provide flexibility in accessing various image
        formats.
    .attr : list
        A list of other attributes set by `**kwargs` to `__init__()`.
    ._1d : dict
        The 1D views of `.image`, `.file`, `._ext`, and all listed in
        `.attr`.
    """
    def __init__(self, im, loader=None, **kwargs):
        """
        Parameters
        ----------
        im : string, string sequence, array-like numbers of shape[..., N, M]
            File name(s) if string or string sequence.  Images must be
            stored in fits files with the extension specified by `ext`.
            Image(s) if array-like numbers.  Has to be 2D or high
            dimension.  The last two dimension are the shape of images
            (NxM).
        ext : int, str, or sequence of them, optional
            The extension of FITS files that stores the images to be
            loaded.  If sequence, then it must have the same shape as
            input file names.
        loader : function, optional
            A function to load an image from input file.  It provides
            flexibility in accessing various image formats.  The call
            signature of the loader is
                im = loader(input, **kwargs)
            where input is a string of file name, and it returns an image
            in a 2D array.
            If `loader` == `None`, then a FITS file will be assumed.
        **kwargs : int or sequence of int, optional
            Any other arguments needed.  If a scaler, it is assumed to
            be the same for all images.  If sequences, it must have the
            same shape as input file names or images.
        """
        # process input images/files
        im = np.array(im)
        if im.dtype.kind in ['S', 'U']:
            # image file name
            if im.ndim == 0:
                self.file = np.array([im])
            else:
                self.file = im
            self.image = None
            self._shape = self.file.shape
            self._size = self.file.size
        elif im.dtype.kind in ['i', 'u', 'f']:
            # image array
            if im.ndim < 2:
                raise ValueError('for images, `.ndim` must be >=2')
            if im.ndim == 2:
                self.image = np.zeros(1, dtype='O')
                self.image[0] = im
            else:
                self.image = np.zeros(im.shape[:-2], dtype='O')
                _from = im.reshape(-1, im.shape[-2], im.shape[-1])
                _to = self.image.reshape(-1)
                for i in range(len(_from)):
                    _to[i] = _from[i]
            self.file = None
            self._shape = self.image.shape
            self._size = self.image.size
        elif im.dtype.kind == 'O':
            types = [x.dtype.kind in ['i', 'u', 'f'] for x in im.reshape(-1)]
            if not np.all(types):
                raise ValueError('unrecognized image type')
            self.image = np.zeros(im.shape, dtype='O')
            _from = im.reshape(-1)
            _to = self.image.reshape(-1)
            for i in range(len(_from)):
                _to[i] = _from[i]
            self.file = None
            self._shape = self.image.shape
            self._size = self.image.size
        else:
            raise ValueError('unrecognized input type')
        # other keywrods
        self.attr = []
        if len(kwargs) > 0:
            for k, v in kwargs.items():
                n = '_'+k
                self.attr.append(n)
                if isinstance(v, str) or (not hasattr(v, '__iter__')):
                    setattr(self, n, v)
                else:
                    if np.asarray(v).shape != self._shape:
                        raise ValueError('invalide shape for kwargs {}: '
                                'expect '. format(k) + str(self._shape) + \
                                ', got ' + str(np.asarray(v).shape))
                    setattr(self, n, np.asarray(v))
        self.loader = loader
        # generate flat views
        self._generate_flat_views()

    def _generate_flat_views(self):
        self._1d = {}
        self._1d['image'] = None if (self.image is None) else \
                    self.image.reshape(-1)
        self._1d['file'] = None if (self.file is None) else \
                    self.file.reshape(-1)
        for k in self.attr:
            v = getattr(self, k)
            if isinstance(v, np.ndarray):
                self._1d[k] = v.reshape(-1)
            else:
                self._1d[k] = np.array([v] * self._size)

    def _load_image(self, i):
        """Load the ith image in flattened file name list
        """
        if self.image is not None and self._1d['image'][i] is not None:
            return
        if self.file is None:
            raise ValueError('no input file specified')
        if self.image is None:
            self.image = np.zeros_like(self.file, dtype='O')
            self.image[:] = None
            self._1d['image'] = self.image.reshape(-1)
        if self.loader is None:
            ext = self._1d['_ext'][i] if '_ext' in self._1d.keys() else 0
            img = fits.open(self._1d['file'][i])[ext].data
            if img is None:
                warnings.warn("empty fits extension {} in file {}".format(
                    ext, self._1d['file'][i]))
            self._1d['image'][i] = img
        else:
            self._1d['image'][i] = self.loader(self._1d['file'][i])

    def _ravel_indices(self, index):
        """Convert index to 1d index

        Parameters
        ----------
        index : None, or int, slice, list of int, or tuple of them
            If `None`, return 1d indices of all items
            If int, slice, or list of int, then specify the 1d index
                (index of flattened `._1d['file']` or `._1d['image']`)
            If tuple, then specify the multi-dimentional index.

        Return
        ------
        index1d : 1d array or tuple of 1d arrays
            The index of flattened `._1d['file']` or `._1d['image']`
        """
        if index is None:
            _index = np.r_[range(self._size)]
        elif isinstance(index, tuple):
            # multi-dimensional indices
            _index = ()
            for i, ind in enumerate(index):
                if isinstance(ind, slice) and (ind.stop is None):
                    ind = slice(ind.start, self._shape[i], ind.step)
                _index = _index + (np.r_[ind], )
            len_index = len(_index)
            if len_index < len(self._shape):
                for i in range(len(self._shape) - len_index):
                    _index = _index + (np.r_[range(self._shape[len_index+i])],)
            _index = np.ravel_multi_index(_index, self._shape)
            if not hasattr(_index, '__iter__'):
                _index = [_index]
        else:
            # 1d indices
            _index = np.r_[index]
        return _index

    @property
    def shape(self):
        return self._shape

    @property
    def size(self):
        return self._size

    def to_table(self):
        cols = []
        for k in self.attr:
            cols.append(table.Column(self._1d[k], name=k.strip('_')))
        if self.file is not None:
            cols.insert(0, table.Column(self._1d['file'], name='file'))
        if all([getattr(c, 'unit', None) is None for c in cols]):
            out = table.Table(cols)
        else:
            out = table.QTable(cols)
        return out

    def write(self, outfile, format=None, save_images=False, **kwargs):
        """Write centers to output file

        Parameters
        ----------
        outfile : str
            Output file name
        format : [None, 'ascii', 'fits'], optional
            Format of output file.
            If `None`, then the format is implied by the extension of
            `outfile`.  If no extension or extension not recognized,
            then an error will be issued.
            If 'ascii', then the centers are flattened if
            `self.file.ndim` > 1, and saved to an ASCII table.  The
            first column is either image file names if provided, or
            image indices otherwise.  The second and third columns are
            centers (x, y), with headings 'xc', 'yc'.  The last column
            is 'status', with value 1 means center is measured successfully
            and 0 means center is not measured.
            If 'fits', then the centers are saved in a FITS array in the
            primary extension, status in the first extension, and the
            image file names in a binary table in the second extension
            if provided.
        save_images : bool, optional
            If `True`, then if `.file` is None and images are provided,
            then save images to a FITS file named
            `'{}_images.fits'.format(outfile)`.
        **kwargs : dict, optional
            Other keyword arguments accepted by the `astropy.io.ascii.write`
            or `astropy.io.fits.HDUList.writeto`.

        The data will be organized in a table and saved in either an ASCII
        file or a FITS file.  The table will include file names (if
        available), extension number (if applicable), and all information
        supplied by **kwargs.  If the image set is a multi-dimensional
        array, then the dimension and shape of the set is also saved in
        the FITS binary table headers if the output is a FITS file.  If
        the output is an ASCII file, then the shape information of image
        set will be discarded and all data saved in flat arrays.
        """
        out = self.to_table()
        if format is None:
            format = ''
            ext = os.path.splitext(outfile)[1].lower()
            if ext in ['.fits', '.fit']:
                format = 'fits'
            elif ext in ['.csv', '.tab', '.ecsv']:
                format = 'ascii'
        if format == 'ascii':
            out.write(outfile, **kwargs)
        elif format == 'fits':
            outfits = fits.HDUList(fits.PrimaryHDU())
            tblhdu = fits.BinTableHDU(out, name='info')
            tblhdu.header['ndim'] = len(self._shape)
            for i in range(len(self._shape)):
                tblhdu.header['axis{}'.format(i)] = self._shape[i]
            outfits.append(tblhdu)
            outfits.writeto(outfile, **kwargs)
        else:
            raise ValueError('unrecognized output format')
        if save_images:
            if self.file is None:
                hdu0 = fits.PrimaryHDU()
                hdu0.header['ndim'] = len(self._shape)
                for i in range(len(self._shape)):
                    hdu0.header['axis{}'.format(i)] = self._shape[i]
                hdulist = fits.HDUList([hdu0])
                for im in self._1d['image']:
                    hdulist.append(fits.ImageHDU(im))
                outname = '{}_images.fits'.format(os.path.splitext(outfile)[0])
                overwrite = kwargs.pop('overwrite', False)
                hdulist.writeto(outname, overwrite=overwrite)

    def read(self, infile, format=None, **kwargs):
        """Read centers from input file

        Parameters
        ----------
        infile : str
            Input file name
        format : [None, 'ascii', 'fits'], optional
            Format of input file.
            If `None`, then the format is implied by the extension of
            `outfile`.  If no extension or extension not recognized,
            then an error will be issued.
            If 'ascii':  If file names are available from the input file,
            then they will replace whatever in `.file` attribute, and
            any loaded images will be cleared.  The centers and status
            will be reshaped to the same shape as `.file` or `.image`.
            If 'fits', then the centers and status will be loaded, and
            if file names are avaialble from input file, then they will
            replace whatever in `.file` attribute, and reshaped to the
            same shape as centers and status.  Any loaded images will
            be cleared in this case.
        **kwargs : dict, optional
            Other keyword arguments accepted by the `astropy.io.ascii.read`.
        """
        if format is None:
            format = ''
            ext = os.path.splitext(infile)[1].lower()
            if ext in ['.fits', '.fit']:
                format = 'fits'
            elif ext in ['.csv', '.tab']:
                format = 'ascii'
        if format == 'ascii':
            intable = ascii.read(infile, **kwargs)
            ndim = 0
        elif format == 'fits':
            with fits.open(infile) as _f:
                intable = table.Table(_f['info'].data)
                ndim = _f['info'].header['ndim']
                shape = ()
                for i in range(ndim):
                    shape = shape + (_f['info'].header['axis{}'.format(i)],)
        else:
            raise ValueError('unrecognized input format')
        keys = intable.keys()
        if 'file' in keys:
            self.file = np.array(intable['file'])
            keys.remove('file')
            self.image = None
        else:
            imgfile = '{}_images.fits'.format(os.path.splitext(infile)[0])
            if not os.path.isfile(imgfile):
                raise IOError('input image file not found')
            self.file = None
            # load image
            with fits.open(imgfile) as _f:
                ndim = _f[0].header['ndim']
                shape = ()
                for i in range(ndim):
                    shape = shape + (_f[0].header['axis{}'.format(i)],)
                self.image = np.zeros(shape, dtype='O')
                image1d = self.image.reshape(-1)
                for i in range(len(_f)-1):
                    image1d[i] = _f[i+1].data
        self.attr = []
        for k in keys:
            n = '_' + k
            self.attr.append(n)
            setattr(self, n, np.array(intable[k]))
        # adjust shape
        if ndim == 0:
            self._shape = len(intable),
            self._size = self._shape[0]
        else:
            self._shape = shape
            self._size = int(np.array(shape).prod())
        if self.file is not None:
            self.file = self.file.reshape(self._shape)
        # process attributes
        keys = self.attr
        for k in keys:
            v = getattr(self, k)
            if np.all(v == v[0]):
                setattr(self, k, v[0])
        # generate flat view
        self._generate_flat_views()

    @classmethod
    def from_fits(cls, infile, loader=None):
        obj = cls('')
        obj.read(infile, format='fits')
        obj.loader = loader
        return obj

class CollectFITSHeaderInfo(ImageSet):
    """Collect FITS header information

    Class can be initialized with `fields` keyword that specify the FITS keyword
    and corresponding FITS extension (number or name):

    >>> fields = [('obs-date', 0), ('obs-time', 0), ('exptime', 0)]
    >>> info = CollectFITSHeaderInfo(files, fields=fields)

    Alternatively, for a specific type of image, a subclass can be defined to
    pre-define the fields to be collected:

    >>> class CollectObsParams(CollectFITSHeaderInfo):
    >>>     fields = [('obs-date', 0), ('obs-time', 0), ('exptime', 0)]
    >>>
    >>> info = CollectObsParams()
    """
    def __init__(self, *args, fields=None, **kwargs):
        """
        fields : array like of (str, int or str) or (str, int or str, str)
            For each line in the array:
                str : FITS header keyword to be collected
                int or str : FITS extension for this keyword
                str : If exist, the unit of the value
        """
        if 'loader' in kwargs.keys():
            warnings.warn("`loader` keyword is ignored.  Only FITS files "
                         "are accepted")
            _ = kwargs.pop('loader')
        super().__init__(*args, **kwargs)
        if fields is not None:
            self.fields = fields
        if not hasattr(self, 'fields'):
            self.fields = None

    def collect(self, verbose=True):
        """Collect header information"""
        if self.fields is None:
            if verbose:
                print('Fields not specified.  No information is collected.')
            return
        keys = ['_' + x[0] for x in self.fields]
        self.attr.extend(keys)
        for k in keys:
            setattr(self, k, [])
        for i in range(self._size):
            with fits.open(self._1d['file'][i]) as f_:
                for line in self.fields:
                    k = line[0]
                    e = line[1]
                    v = f_[e].header[k]
                    if len(line) > 2:
                        v = v * u.Unit(line[2])
                    getattr(self, '_'+k).append(v)
        for k in keys:
            v = getattr(self, k)
            if hasattr(v[0], 'unit'):
                v = u.Quantity(v)
            else:
                v = np.array(v)
            setattr(self, k, v)
        self._generate_flat_views()

In [ ]:
# extract image mata data from fits headers

#from jylipy.image import CollectFITSHeaderInfo

# fits keys to collect
keys = ['NAXIS1', 'NAXIS2', 'EXPTIME', 'OBSID', 'STRTSCLK', 'STOPSCLK','VISTEMP', 'CCD','TARGETID','ALLOCBLK','BSRAMID', 'BSDCMID']# 'source', 'binmode', 'expmode', 'exptime',
       # 'spcutcal', 'spcbrra', 'spcbrdec']
    #keys = ['NAXIS1', 'NAXIS2', 'EXPTIME', 'OBSID', 'STRTSCLK', 'STOPSCLK','CCDT1', 'EXPMODE','TARGETID','CFORMAT','BSRAMID', 'BSDCMID']

fields = [(x, 0) for x in keys]
asp = CollectFITSHeaderInfo(rawfile, fields=fields)
asp.collect()
asp.write('/meta_raw.csv', overwrite=True)